In [1]:
# Importing Neo4j required elements
!pip install neo4j
from neo4j import GraphDatabase

class App:
# Class Neo4j driver

    def __init__(self, uri, user, password):
        #Instantiation of a Neo4j driver
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        #Do not forget to close the driver at the end of the script
        self.driver.close()

    def import_csv(self,file_path,separator:str,label:str):
        #CSV file import with fieldterminator=separator and creation of nodes with label=label
        with self.driver.session(database="neo4j") as session:
            query = (" load csv with headers from $file_path as row fieldterminator '"+separator+"' "
                    " create (:"+label+" {id: row.ID, nom: row.Nom_du_POI , "
                    " latitude:tofloat(row.Latitude), longitude:tofloat(row.Longitude), "
                    " description: row.Description, adresse: row.Adresse_postale, commune: row.commune, "
                    " code: row.code_postal, categorie: row.category_0, contact: row.Contacts_du_POI }) "
                    )
            result = session.run(query,file_path=file_path)

    def delete_node_all(self):
        #Deletion of all nodes of the graph
        with self.driver.session(database="neo4j") as session:
            query = ( "MATCH (p) DETACH DELETE p")
            result = session.run(query)
 
    def delete_node(self, label:str):
        #Deletion of all nodes with label=label
        with self.driver.session(database="neo4j") as session:
            query = ( "MATCH (p:"+label+") DETACH DELETE p")
            result = session.run(query)   
    
    def create_node_coordinates(self,label:str):
        #Add location attribute to each node as point
         with self.driver.session(database="neo4j") as session:
            query = ( "MATCH (p:"+label+") "
                        " SET p.location = point({latitude: p.latitude, longitude: p.longitude})"
                    )
            result = session.run(query)    


    def create_space_relationship(self,label:str,city="All",max_distance=500):
        # Create non directionnal relationship with label=SPACE and attribute=distance:kms
        # between all nodes with label=label within commune=city, distant by no more than
        # max_distance kms
        if city!="All":
            with self.driver.session(database="neo4j") as session:
                query = ( " MATCH(p1 :"+label+" {commune: '"+city+"'}) "
                        " MATCH(p2 :"+label+" {commune : '"+city+"'}) "
                        " WHERE point.distance(p1.location,p2.location)/100 < $max_distance AND p1.nom<>p2.nom "
                        " CREATE (p1)-[:SPACE {distance:point.distance(p1.location,p2.location)/100}]->(p2) "
                        )
                result = session.run(query,max_distance=max_distance) 
        else:
            with self.driver.session(database="neo4j") as session:
                query = ( " MATCH(p1 :"+label+") "
                        " MATCH(p2 :"+label+") "
                        " WHERE point.distance(p1.location,p2.location)/100 < $max_distance AND p1.nom<>p2.nom "
                        " CREATE (p1)-[:SPACE {distance:point.distance(p1.location,p2.location)/100}]->(p2) "
                        )
                result = session.run(query,max_distance=max_distance) 

    def delete_relationship(self,rel_label:str,node_label:str,city="All"):
        # Delete relationship with label=rel_label between all nodes with label=node_label
        # within commune=city
        if city!="All":
            with self.driver.session(database="neo4j") as session:
                query = ( " MATCH(p1 :"+node_label+" {commune: '"+city+"'}) "
                        " MATCH(p2 :"+node_label+" {commune : '"+city+"'}) "
                        " WHERE p1.nom<>p2.nom "
                        " MATCH(p1)-[r:"+rel_label+"]->(p2)"
                        " DELETE r "
                        )
                result = session.run(query) 
        else:
            with self.driver.session(database="neo4j") as session:
                query = (" MATCH(p1 :"+node_label+") "
                        " MATCH(p2 :"+node_label+") "
                        " WHERE p1.nom<>p2.nom "
                        " MATCH(p1)-[r:"+rel_label+"]->(p2)"
                        " DELETE r "
                        )
                result = session.run(query) 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.0/171.0 KB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Discarding https://files.pythonhosted.org/packages/d3/b7/f1f8ffdb12d7e10d31b0a23bc2d8a78ad4659474eda2a62f023717df2ba1/neo4j-5.5.0.tar.gz#sha256=2632386380b2ebb7d6a80e4186899ef342ef0507601c65e200696f13742046b8 (from https://pypi.org/simple/neo4j/) (requires-python:>=3.7): Requested unknown from https://files.pythonhosted.org/packages/d3/b7/f1f8ffdb12d7e10d31b0a23bc2d8a78ad4659474eda2a62f023717df2ba1/neo4j-5.5.0.tar.gz#sha256=2632386380b2ebb7d6a80e4186899ef342ef0507601c65e200696f13742046b8 has inconsistent name: filename has 'neo4j', but metadata has 'unknown'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.5/162.5 KB 17.7 MB/s eta 0:00

In [3]:
#Creating the Neo4j drive
uri = "neo4j+s://096fe035.databases.neo4j.io"
user = "neo4j"
password = "vD5Yq0ufk4CayM_7CRD7R1QZrjpnHr4_Yv1Y4rBolzU"
app = App(uri, user, password)

In [4]:
# Creating the POI nodes
file_path_POI2="https://drive.google.com/uc?id=1eAvapPEySpUCGHY1SyAHRwKPC7ndTDD5&confirm=t"
app.import_csv(file_path_POI2,separator=",",label="POI")

In [5]:
# Creating the food nodes
file_path_food="https://drive.google.com/uc?id=1-Etk8NuirWunJ9lxsmC0e6Qup6S6gujq&confirm=t"
app.import_csv(file_path_food,separator=",",label="Food")

In [6]:
# Creating the POI coordinates and creating SPACE relationship with km distance attribute
# for POI nodes distant by less than 5 kms
app.create_node_coordinates(label="POI")
app.create_space_relationship(label="POI",city="All",max_distance=5)

In [7]:
# Creating the food coordinates and creating SPACE relationship with km distance attribute
# for food nodes within commune = Saint-Pierre distant by less than 5 kms
app.create_node_coordinates(label="Food")
app.create_space_relationship(label="Food",city="Saint-Pierre",max_distance=5)

In [9]:
# Deleting SPACE relationship on nodes POI & food
app.delete_relationship(rel_label="SPACE",node_label="POI",city="All")
app.delete_relationship(rel_label="SPACE",node_label="Food",city="All")

In [10]:
# Deleting all nodes
app.delete_node_all()

In [11]:
# Closing Noe4j drive
app.close()

# Nouvelle section